# 📊 **QA Script for Population Projections**

## 📝 **Introduction**
This notebook aims to identify outliers in the GLA population projection data. The analysis involves loading the dataset, preprocessing the data, defining utility functions, performing outlier detection, and presenting the results through visualisations.

### 🎯 **Goals**
The analysis will focus on the following objectives:
- **Load and preprocess** the population projections dataset.
- Define utility functions.
- Perform **basic checks** on the dataset:
  - Range of years covered.
  - Missing values.
  - Duplicates.
  - Descriptive statistics.
  - Breakdown by components.
  - Detecting negative values.
  - Age group ranges.
- **Outlier Detection** over time for each component:
  - Identify outliers using **Z-scores** and **Robust Z-scores**.
  - Analyse by **component**, **ward**, and **borough**.
  - Handle **infinite values** separately.
- **Total Population Outliers**:
  - Use Z-scores and Robust Z-scores for comparison.
  - Perform **cross-sectional comparisons**: Examine changes between boroughs and wards for a given year.
  - Conduct **temporal comparisons**: Measure percentage changes between years for both boroughs and wards.
  - Handle **infinite values** separately.
- **Gender Outliers**:
  - Investigate abnormal **gender ratios**.
  - Analyse by component.
  - Adjust the **outlier standard deviation thresholds** as needed based on different components.
- **Key Visualisations**:
  - Display the distribution of components.
  - Group data by **age ranges**.
  - Visualise **yearly totals**.
  - Show yearly total trends over time, broken down by components.
  - Create **population pyramids**.
- **Collate Outliers**: Summarise and determine the key outlier rows.

---

## 📂 **Datasets**
How should the dataset be structured?

### Dataset 1
The functions are designed to take datasets in the form produced by the GLA Population Projection Workflow. In the current iteration of this workflow, the first dataset contains several key columns and components as outlined below:

#### Main Columns
- **gss_code**: Borough geocode (e.g., GSS code).
- **la_name**: Local Authority name.
- **Year**: The year of the population data.
- **Sex**: The gender (male/female).
- **Age**: The age group or specific age.
- **Value**: Count of the population in the given category.
- **gss_code_ward**: Geocode for the ward.
- **ward_name**: Name of the ward.

#### Components Column:
This column includes specific population-related metrics:
- **net-flow**: Population migration inflow minus migration outflow.
- **popn**: Total population.
- **birth**: Number of births.
- **deaths**: Number of deaths.

### Dataset 2

The second dataset that is examined for outliers is the average household size (ahs) dataset. This is analysed using the find_ahs_outliers_with_context function.

The dataset just contains average household size from 2021 until 2050 by ward

### 


---

## 🛠️ **Structure**
1. [**Load Data** the population projections dataset.](#load-data)
2. [**Define Utility Functions** for effective use.](#define-utility-functions)
3. [**Process Data**](#process-data)
4. [**Basic Checks** on the dataset.](#basic-checks)
5. [**Population Consistency Over Time** for each component.](#population-consistency-over-time)
6. [**Total Population Outliers**](#total-population-outliers)
7. [**Gender Outliers**](#gender-outliers)
8. [**Key Visualisations**](#key-visualisations)
9. [**Collate Outliers** to determine key outlier rows.](#collate-outliers)
10. [**Average Household Size** Outliers.](#average-household-size_outliers)



## Load Data
This section will cover how to load and preprocess the dataset.

---


In [5]:
import pandas as pd
import numpy as np
import pyreadr
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px
from typing import Dict
from scipy.stats import zscore
from scipy.stats import skew


In [2]:
os.chdir(r'C:\Users\user\Documents\population_data\combined_10yr_central_fert')
combined_10yr_fert = pd.read_csv('combined_10yr_central_fert.csv').iloc[:, 1:]

C:\Users\user\AppData\Local\Temp\ipykernel_27328\1627369184.py:2: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_10yr_fert = pd.read_csv('combined_10yr_central_fert.csv').iloc[:, 1:]


In [159]:
combined_10yr_fert

,gss_code,la_name,year,sex,age,value,component,gss_code_ward,ward_name
0,E09000001,City of London,2012.0,female,0.0,32.0,births,NaN,NaN
1,E09000001,City of London,2012.0,male,0.0,24.0,births,NaN,NaN
2,E09000001,City of London,2013.0,female,0.0,36.0,births,NaN,NaN
3,E09000001,City of London,2013.0,male,0.0,35.0,births,NaN,NaN
4,E09000001,City of London,2014.0,female,0.0,26.0,births,NaN,NaN
...,...,...,...,...,...,...,...,...,...
15428057,E09000033,Westminster,2050.0,male,86.0,16.9,popn,E05013809,Westbourne
15428058,E09000033,Westminster,2050.0,male,87.0,13.3,popn,E05013809,Westbourne
15428059,E09000033,Westminster,2050.0,male,88.0,11.0,popn,E05013809,Westbourne
15428060,E09000033,Westminster,2050.0,male,89.0,9.9,popn,E05013809,Westbourne


## Define Utility Functions
Define utility functions that will be used for various parts of the analysis.

---

In [160]:
def view_descriptive_statistics(df, columns):
    """
    Calculate descriptive statistics, including mean, median, and mode, for specified columns in a DataFrame.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    columns (list): List of columns for which to calculate the statistics.
    
    Returns:
    pd.DataFrame: DataFrame containing the descriptive statistics including median and mode.
    """
    # Get descriptive statistics using describe()
    descriptive_stats = df[columns].describe()

    # Calculate median for each column
    median = df[columns].median()

    # Calculate mode for each column (in case of multiple modes, take the first one)
    mode = df[columns].mode().iloc[0]

    # Add median and mode to the descriptive statistics DataFrame
    descriptive_stats.loc['median'] = median
    descriptive_stats.loc['mode'] = mode

    # Return the combined descriptive statistics
    return descriptive_stats

In [161]:
def create_age_bins(df, age_column='age', bins=None, labels=None):
    """
    Create age bins for the specified age column in the given DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the age data.
    age_column (str): The name of the column containing age data. Default is 'age'.
    bins (list): A list of bin edges for categorizing ages. Default is None.
    labels (list): A list of labels for the bins. Default is None.

    Returns:
    pd.DataFrame: The DataFrame with a new 'age' column containing binned age data.
    """
    
    # If bins and labels are not provided, set default values
    if bins is None:
        bins = [-1, 18, 30, 40, 50, 60, 70, 80, 89, 90]
    
    if labels is None:
        labels = ['0-18', '19-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-89', '90+']
    
    # Create a copy of the original DataFrame to avoid modifying it directly
    binned_df = df.copy()
    
    # Create age bins
    binned_df[age_column] = pd.cut(binned_df[age_column], bins=bins, labels=labels)
    
    return binned_df

# Example usage:
# combined_10yr_fert_agebins = create_age_bins(combined_10yr_fert)


In [162]:
def calculate_zscores_and_find_outliers(df, component_columns, handle_inf=True, Geography='borough', z_score_threshold=2, For_population_totals=False, population_analysis_type='cross-sectional'):
    """
    Computes z-scores or robust z-scores (depending on distribution) for the respective columns,
    and returns DataFrames containing outliers for either the component columns (handle_inf=True) 
    or the percentage change columns (handle_inf=False).

    Parameters:
    df (pd.DataFrame): The input DataFrame containing population data component value columns.
    component_columns (str or list): A single column name or a list of column names to be analysed.
    handle_inf (bool): If True, uses the component columns to determine outliers. 
                       If False, uses percentage change columns to determine outliers.
    Geography (str): Specifies whether to group by 'borough' (using 'gss_code') or 'ward' (using 'gss_code_ward').
                     Default is 'borough'.
    For_population_totals (bool): If True, calculates total population sums and percentage changes before proceeding 
                                  to z-score and outlier analysis.
    population_analysis_type (str): Specifies whether to do 'cross-sectional' or 'temporal' analysis for population totals.
                                    Default is 'cross-sectional'.
    z_score_threshold (float or int): The threshold to consider as an outlier based on the z-score. Default is 2.

    Returns:
    dict: A dictionary containing DataFrames with outliers for each respective column based on the z-score threshold.
    """
    
    # If a single column name is provided as a string, convert it to a list
    if isinstance(component_columns, str):
        component_columns = [component_columns]

    outliers_dict = {}
    z_score_type = {}  # Dictionary to store which method was used

    # Grouping and pivot based on the Geography parameter
    if Geography == 'borough':
        geo_column = 'gss_code'
    elif Geography == 'ward':
        geo_column = 'gss_code_ward'
    else:
        raise ValueError("Geography must be either 'borough' or 'ward'.")

    # Automatically create pct_change_columns based on component_columns
    pct_change_columns = [f"{col}_pct_change" for col in component_columns]

    #Calculate the percentage change for the component columns
    df[pct_change_columns] = df.groupby([geo_column, 'sex', 'age'])[component_columns].pct_change().abs()
    #pivoted = df.pivot_table(index=[geo_column, 'sex', 'age', 'year'], columns='component', values='value').reset_index()

    # If For_population_totals is True, calculate population totals and percentage changes
    if For_population_totals:
        # Filter the DataFrame to only keep rows where the 'component' is 'popn'
        # popn_df = pivoted[pivoted['component'] == 'popn']
        
        # Pivot the data to get 'popn' values by geography, sex, age, and year
        # pivoted = popn_df.pivot_table(index=[geo_column, 'sex', 'age', 'year'], columns='component', values='value').reset_index()
        
        # Ensure that there's a 'popn' column in the resulting DataFrame
        if 'popn' in df.columns:
            df['popn'] = df['popn']  # Extract the 'popn' column
        else:
            raise ValueError("The 'popn' column is missing after pivoting.")

        # Group by geography and year, and sum the population values, create total population dfs for crossectional and temporal analysis change
        population_sum = df.groupby([geo_column, 'year'])['popn'].sum().reset_index()

        population_sum_time = population_sum.copy()
        population_sum_cross = population_sum.copy()

        if population_analysis_type == 'temporal':
            # Temporally: Calculate the population change over the years for each gss_code or ward
            population_sum_time['popn_pct_change_temporal'] = population_sum_time.groupby(geo_column)['popn'].pct_change() * 100
        elif population_analysis_type == 'cross-sectional':
            # Cross-sectionally: Compare the population between different gss_code or wards for the same year and compare to the mean
            population_sum_cross['popn_mean'] = population_sum_cross.groupby('year')['popn'].transform('mean')
            population_sum_cross['popn_pct_change_cross'] = ((population_sum_cross['popn'] - population_sum_cross['popn_mean']) / population_sum_cross['popn_mean']) * 100
        else:
            raise ValueError("population_analysis_type must be either 'cross-sectional' or 'temporal'.")

    # Now, decide how to determine outliers based on handle_inf (handling the infinite values e.i. no percentage change value) and pct_change_columns
    if handle_inf:
        # Outliers based on component columns value (not percentage change)
        for comp_col, pct_change_col in zip(component_columns, pct_change_columns):
            if pct_change_col in df.columns:
                # Filter rows where percentage change columns have inf or -inf
                df_filtered = df.replace([np.inf, -np.inf], np.nan).dropna(subset=[comp_col, pct_change_col])

                if not df_filtered.empty:
                    # Check if the column is normally distributed using skewness
                    skewness = df_filtered[comp_col].skew()

                    if abs(skewness) < 0.5:  # If skewness is less than 0.5, use normal Z-score
                        df_filtered['z_score'] = stats.zscore(df_filtered[comp_col])
                        outliers = df_filtered[df_filtered['z_score'].abs() > z_score_threshold]  # Z-score > threshold
                        z_score_type[comp_col] = 'Normal Z-Score'
                        print(f"{comp_col} used Normal Z-Score.")
                    else:
                        # Use Robust Z-score (based on median and MAD) for non-normal distribution
                        median = df_filtered[comp_col].median()
                        mad = stats.median_abs_deviation(df_filtered[comp_col])
                        df_filtered['robust_z_score'] = (df_filtered[comp_col] - median) / mad
                        outliers = df_filtered[df_filtered['robust_z_score'].abs() > z_score_threshold]  # Robust Z-score > threshold
                        z_score_type[comp_col] = 'Robust Z-Score'
                        print(f"{comp_col} used Robust Z-Score.")

                    # Store the outliers for this component column
                    outliers_dict[comp_col] = outliers
                else:
                    outliers_dict[comp_col] = pd.DataFrame()  # Return empty DataFrame if no rows found
            else:
                print(f"{comp_col} does not exist in DataFrame")

    else:
        # Outliers based on pct_change_columns
        for pct_change_col in pct_change_columns:
            # For population totals choose the correct DataFrame (from above) based on the selected column
            if population_analysis_type == 'temporal' and 'popn_pct_change_temporal' in pct_change_col:
                df_filtered = population_sum_time  # Use the temporal population data
            elif population_analysis_type == 'cross-sectional' and 'popn_pct_change_cross' in pct_change_col:
                df_filtered = population_sum_cross  # Use the cross-sectional population data
            else:
                df_filtered = df  # Default to the original df if other percentage columns are provided

            #pct column for component column zscores
            if pct_change_col in df_filtered.columns:
                # Replace inf and -inf with NaN and work on the entire DataFrame after cleaning
                df_filtered = df_filtered.replace([np.inf, -np.inf], np.nan).dropna(subset=[pct_change_col])

                if not df_filtered.empty:
                    # Check if the column is normally distributed using skewness
                    skewness = df_filtered[pct_change_col].skew()

                    if abs(skewness) < 0.5:  # If skewness is less than 0.5, use normal Z-score
                        df_filtered['z_score'] = stats.zscore(df_filtered[pct_change_col])
                        outliers = df_filtered[df_filtered['z_score'].abs() > z_score_threshold]  # Z-score > threshold
                        z_score_type[pct_change_col] = 'Normal Z-Score'
                        print(f"{pct_change_col} used Normal Z-Score.")
                    else:
                        # Use Robust Z-score (based on median and MAD) for non-normal distribution
                        median = df_filtered[pct_change_col].median()
                        mad = stats.median_abs_deviation(df_filtered[pct_change_col])
                        df_filtered['robust_z_score'] = (df_filtered[pct_change_col] - median) / mad
                        outliers = df_filtered[df_filtered['robust_z_score'].abs() > z_score_threshold]  # Robust Z-score > threshold
                        z_score_type[pct_change_col] = 'Robust Z-Score'
                        print(f"{pct_change_col} used Robust Z-Score.")

                    # Store the outliers for this percentage change column
                    outliers_dict[pct_change_col] = outliers
                else:
                    outliers_dict[pct_change_col] = pd.DataFrame()  # Return empty DataFrame if no rows found
            else:
                print(f"{pct_change_col} does not exist in DataFrame")

    # Sort each DataFrame in outliers_dict by 'z_score' or 'robust_z_score' in descending order in one line
    for key in outliers_dict.keys():
        outliers_dict[key] = outliers_dict[key].sort_values(by='z_score' if 'z_score' in outliers_dict[key].columns else 'robust_z_score', ascending=False)

    return outliers_dict  # Returning z_score_type for further use if needed


In [163]:
def robust_z_score(series):
    """Calculate robust z-score using median and MAD."""
    median = series.median()
    mad = (series - median).abs().median()
    return 0.6745 * (series - median) / mad if mad != 0 else np.zeros_like(series)

def gender_outliers(df, component_columns, geography='borough', outlier_std={'births': 2, 'deaths': 5, 'netflow': 2, 'popn': 5}):
    """
    Processes gender data for either wards or boroughs, and calculates outliers for specified components.
    Uses either z-score or robust z-score based on skewness.

    Parameters:
    - df: pandas DataFrame containing the raw data
    - component_columns: list or single component column name(s) for which ratios and outliers need to be calculated
    - geography: str, either 'ward' or 'borough', default is 'borough'
    - outlier_std: dict specifying how many standard deviations to use for each component's threshold calculation.
    
    Returns:
    - outliers_dict: dictionary of outlier DataFrames for each component
    """

    # Check geography type and set index columns accordingly
    if geography == 'ward':
        geo_col = 'gss_code_ward'
    else:
        geo_col = 'gss_code'
    
    # Step 1: Create the pivot table
    gender_pivot = df.pivot_table(
        index=[geo_col, 'year', 'age', 'component'],  # Geography column and other grouping columns
        columns='sex',                                # Columns for sex (male, female)
        values='value',                               # Values (count of males and females)
        aggfunc='sum'                                 # Aggregation function (sum)
    ).reset_index()
    
    # Step 2: Calculate the ratio of females to males
    gender_pivot['ratio_female_to_male'] = gender_pivot['female'] / gender_pivot['male']
    
    # Handle division by zero and missing values
    gender_pivot['ratio_female_to_male'].replace([float('inf'), -float('inf')], pd.NA, inplace=True)
    gender_pivot['ratio_female_to_male'].fillna(np.nan, inplace=True)
    
    # Step 3: Pivot again to spread component values into columns
    gender_pivot = gender_pivot.pivot(
        index=[geo_col, 'year', 'age'], 
        columns='component', 
        values='ratio_female_to_male'
    ).reset_index()
    
    # Step 4: Convert the specified component columns to numeric
    for component in component_columns:
        gender_pivot[component] = pd.to_numeric(gender_pivot[component], errors='coerce')
    
    # Step 5: Detect skewness and calculate either z-score or robust z-score for each component
    outliers_dict = {}
    for component in component_columns:
        component_data = gender_pivot[component].dropna()

        # Calculate skewness
        component_skewness = skew(component_data)
        
        # Decide whether to use z-score or robust z-score
        if abs(component_skewness) > 0.5:
            # Use robust z-score if skewness is high
            z_column = 'robust_z_score'
            gender_pivot[z_column] = robust_z_score(gender_pivot[component])
        else:
            # Use standard z-score if skewness is low
            z_column = 'z_score'
            gender_pivot[z_column] = zscore(gender_pivot[component], nan_policy='omit')
        
        # Set the outlier threshold based on user input (default 2, 5, etc. depending on the component)
        threshold = outlier_std.get(component, 2)
        
        # Step 6: Identify outliers where absolute z-score is greater than the threshold
        outliers = gender_pivot[gender_pivot[z_column].abs() > threshold]
        
        # Step 7: Keep only relevant columns in the output (geo_col, year, age, z_score, and the component itself)
        outliers = outliers[[geo_col, 'year', 'age', component, z_column]]
        
        # Store the outliers DataFrame in the outliers dictionary
        outliers_dict[component] = outliers.reset_index(drop=True)
    
    return outliers_dict



In [164]:
def tally_outliers(outlier_dfs: Dict[str, pd.DataFrame], 
                   tally_by_age: bool = False, 
                   geography: str = 'ward') -> pd.DataFrame:
    """
    Tally occurrences of the same 'gss_code_ward' or 'gss_code', 
    'year', and optionally 'age' across multiple DataFrames.

    Parameters:
    ----------
    outlier_dfs : Dict[str, pd.DataFrame]
        A dictionary where keys are DataFrame names and values are the DataFrames themselves.
        Each DataFrame should contain 'gss_code_ward' or 'gss_code', 'year', and optionally 'age' columns.

    tally_by_age : bool, optional
        If True, the function will tally occurrences based on 'gss_code_ward' or 'gss_code', 'year', and 'age'.
        If False, it will tally based only on 'gss_code_ward' or 'gss_code' and 'year'. Default is False.

    geography : str, optional
        Specifies the geographical code to use. Set to 'borough' to use 'gss_code' 
        or 'ward' to use 'gss_code_ward'. Default is 'ward'.

    Returns:
    -------
    pd.DataFrame
        A DataFrame with counts of occurrences for each combination of 
        'gss_code_ward' or 'gss_code', 'year', and 'age' (if applicable), 
        along with a total count across all DataFrames.
    """
    
    # Validate input
    if not isinstance(outlier_dfs, dict):
        raise ValueError("Expected 'outlier_dfs' to be a dictionary of DataFrames.")
    
    if geography not in ['borough', 'ward']:
        raise ValueError("geography must be either 'borough' or 'ward'.")

    # Initialise an empty DataFrame for the combined tally
    combined_tally = pd.DataFrame()

    # Define the geography column based on the geography argument
    geo_column = 'gss_code' if geography == 'borough' else 'gss_code_ward'

    # Loop through each DataFrame in the outlier_dfs dictionary
    for df_name, df in outlier_dfs.items():  # Unpack tuple here
        if not isinstance(df, pd.DataFrame):
            raise ValueError(f"Expected '{df_name}' to be a pandas DataFrame.")
        
        # Ensure 'age' is consistently typed if tally_by_age is True
        if tally_by_age and 'age' in df.columns:
            df['age'] = df['age'].astype(str)

        # Select the required columns based on the tally_by_age flag
        group_cols = [geo_column, 'year'] + (['age'] if tally_by_age and 'age' in df.columns else [])

        # Ensure selected columns exist in the DataFrame
        missing_cols = [col for col in group_cols if col not in df.columns]
        if missing_cols:
            raise KeyError(f"The DataFrame '{df_name}' is missing columns: {missing_cols}")

        # Remove duplicate rows based on the selected grouping columns
        subset = df[group_cols].drop_duplicates()
        
        # Add a count column for tallying occurrences
        subset['count'] = 1

        # Aggregate the counts by the grouping columns
        grouped = subset.groupby(group_cols).count().reset_index()

        # Rename the count column to indicate which DataFrame it came from
        grouped.rename(columns={'count': df_name}, inplace=True)

        # Merge the current tally with the combined tally
        if combined_tally.empty:
            combined_tally = grouped
        else:
            combined_tally = pd.merge(combined_tally, grouped, on=group_cols, how='outer')

    # Fill NaN values with 0 (indicating no occurrences in that DataFrame)
    combined_tally.fillna(0, inplace=True)

    # Calculate the total tally across all DataFrame columns
    tally_columns = combined_tally.columns.difference(group_cols)
    combined_tally['total'] = combined_tally[tally_columns].sum(axis=1)

    # Sort the DataFrame by the total count in descending order
    combined_tally.sort_values(by='total', ascending=False, inplace=True)

    return combined_tally




In [165]:
def tally_outliers(outlier_dfs: Dict[str, pd.DataFrame], 
                   tally_by_age: bool = False, 
                   geography: str = 'ward') -> pd.DataFrame:
    """
    Tally occurrences of the same 'gss_code_ward' or 'gss_code', 
    'year', and optionally 'age' across multiple DataFrames, 
    and merge with the total outlier score from 'robust_z_score' or 'z_score'.

    Parameters:
    ----------
    outlier_dfs : Dict[str, pd.DataFrame]
        A dictionary where keys are DataFrame names and values are the DataFrames themselves.
        Each DataFrame should contain 'gss_code_ward' or 'gss_code', 'year', and optionally 'age' columns.

    tally_by_age : bool, optional
        If True, the function will tally occurrences based on 'gss_code_ward' or 'gss_code', 'year', and 'age'.
        If False, it will tally based only on 'gss_code_ward' or 'gss_code' and 'year'. Default is False.

    geography : str, optional
        Specifies the geographical code to use. Set to 'borough' to use 'gss_code' 
        or 'ward' to use 'gss_code_ward'. Default is 'ward'.

    Returns:
    -------
    pd.DataFrame
        A DataFrame with counts of occurrences for each combination of 
        'gss_code_ward' or 'gss_code', 'year', and 'age' (if applicable), 
        along with a total count across all DataFrames and the total outlier score.
    """
    
    # Validate input
    if not isinstance(outlier_dfs, dict):
        raise ValueError("Expected 'outlier_dfs' to be a dictionary of DataFrames.")
    
    if geography not in ['borough', 'ward']:
        raise ValueError("geography must be either 'borough' or 'ward'.")

    # Initialise an empty DataFrame for the combined tally
    combined_tally = pd.DataFrame()

    # Define the geography column based on the geography argument
    geo_column = 'gss_code' if geography == 'borough' else 'gss_code_ward'

    # Loop through each DataFrame in the outlier_dfs dictionary
    for df_name, df in outlier_dfs.items():
        if not isinstance(df, pd.DataFrame):
            raise ValueError(f"Expected '{df_name}' to be a pandas DataFrame.")
        
        # Ensure 'age' is consistently typed if tally_by_age is True
        if tally_by_age and 'age' in df.columns:
            df['age'] = df['age'].astype(str)

        # Select the required columns based on the tally_by_age flag
        group_cols = [geo_column, 'year'] + (['age'] if tally_by_age and 'age' in df.columns else [])

        # Ensure selected columns exist in the DataFrame
        missing_cols = [col for col in group_cols if col not in df.columns]
        if missing_cols:
            raise KeyError(f"The DataFrame '{df_name}' is missing columns: {missing_cols}")

        # Remove duplicate rows based on the selected grouping columns
        subset = df[group_cols].drop_duplicates()
        
        # Add a count column for tallying occurrences
        subset['count'] = 1

        # Aggregate the counts by the grouping columns
        grouped = subset.groupby(group_cols).count().reset_index()

        # Rename the count column to indicate which DataFrame it came from
        grouped.rename(columns={'count': df_name}, inplace=True)

        # Merge the current tally with the combined tally
        if combined_tally.empty:
            combined_tally = grouped
        else:
            combined_tally = pd.merge(combined_tally, grouped, on=group_cols, how='outer')

    # Fill NaN values with 0 (indicating no occurrences in that DataFrame)
    combined_tally.fillna(0, inplace=True)

    # Calculate the total tally across all DataFrame columns
    tally_columns = combined_tally.columns.difference(group_cols)
    combined_tally['total'] = combined_tally[tally_columns].sum(axis=1)

    # Sort the DataFrame by the total count in descending order
    combined_tally.sort_values(by='total', ascending=False, inplace=True)

    # --- New logic to handle multiple score column names ---

    # Concatenate all the DataFrames in the outlier_dfs dictionary
    combined_df = pd.concat(outlier_dfs.values())

    # Check if the DataFrame contains 'robust_z_score' or 'z_score' and create a unified column
    if 'robust_z_score' in combined_df.columns:
        combined_df['outlier_score'] = combined_df['robust_z_score']
    elif 'z_score' in combined_df.columns:
        combined_df['outlier_score'] = combined_df['z_score']
    else:
        raise ValueError("None of the DataFrames contain 'robust_z_score' or 'z_score'.")

    # Group by 'gss_code_ward' (or 'gss_code'), 'year', and 'age', and sum the outlier scores
    group_outliers = combined_df.groupby(
        [geo_column, 'year'] + (['age'] if tally_by_age and 'age' in combined_df.columns else []),
        as_index=False
    ).agg(
        total_outlier_score=('outlier_score', 'sum')
    )

    # Merge the total outlier score with the combined_tally DataFrame
    final_df = pd.merge(
        combined_tally, group_outliers, 
        on=[geo_column, 'year'] + (['age'] if tally_by_age and 'age' in group_outliers.columns else []), 
        how='left'
    )

    return final_df


In [20]:
def find_ahs_outliers_absolute_value(df, z_threshold=3):
    """
    Identifies outliers in the average household size (ahs) dataframe based on z-scores and returns a DataFrame with additional context.
    
    Args:
    - df (pd.DataFrame): DataFrame with household size data. First column should be 'gss_code_ward'.
    - z_threshold (float): Threshold for z-scores to determine outliers. Default is 3.
    
    Returns:
    - pd.DataFrame: DataFrame containing gss_code_ward, year, z_score, outlier value, year_before, year_after,
                    average_for_year (across wards), and average_for_ward (across years).
    """
    
    # Step 1: Calculate z-scores for each year column (excluding 'gss_code_ward')
    z_scores = df.iloc[:, 1:].apply(zscore)

    # Step 2: Identify the outliers (z-scores > z_threshold or z-scores < -z_threshold)
    outliers = z_scores[(z_scores > z_threshold) | (z_scores < -z_threshold)]

    # Step 3: Combine 'gss_code_ward' with outliers to present them in a readable format
    outliers_cleaned = pd.concat([df['gss_code_ward'], outliers], axis=1)

    # Step 4: Drop rows with no outliers
    outliers_cleaned = outliers_cleaned.dropna(how='all', subset=outliers_cleaned.columns[1:])

    # Step 5: Melt the DataFrame to have 'gss_code_ward', 'year', and 'z_score' columns
    outliers_melted = outliers_cleaned.melt(id_vars='gss_code_ward', var_name='year', value_name='z_score')

    # Step 6: Drop NaN values (non-outliers) and sort by z_score in descending order
    outliers_sorted = outliers_melted.dropna().sort_values(by='z_score', ascending=False)

    # Convert the 'year' column to integer for easier manipulation
    outliers_sorted['year'] = outliers_sorted['year'].astype(int)

    # Step 7: Create new columns for the value of AHS for outlier year, previous year, and next year
    outliers_sorted['outlier_value'] = outliers_sorted.apply(
        lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], str(row['year'])].values[0], axis=1)

    outliers_sorted['year_before_value'] = outliers_sorted.apply(
        lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], str(row['year'] - 1)].values[0]
        if str(row['year'] - 1) in df.columns else None, axis=1)

    outliers_sorted['year_after_value'] = outliers_sorted.apply(
        lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], str(row['year'] + 1)].values[0]
        if str(row['year'] + 1) in df.columns else None, axis=1)

    # Step 8: Calculate the average for that year across all gss_code_wards
    outliers_sorted['average_for_year'] = outliers_sorted.apply(
        lambda row: df[str(row['year'])].mean(), axis=1)

    # Step 9: Calculate the average for that gss_code_ward across all years
    outliers_sorted['average_for_ward'] = outliers_sorted.apply(
        lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], df.columns[1:]].mean(axis=1).values[0], axis=1)

    # Step 10: Create the final outlier_sorted_context DataFrame
    outlier_sorted_context = outliers_sorted[['gss_code_ward', 'year', 'z_score', 'outlier_value', 'year_before_value', 'year_after_value', 'average_for_year', 'average_for_ward']]

    return outlier_sorted_context

# Example usage:
#outlier_sorted_context = find_outliers_with_context(average_household_size, z_threshold=3)

# If you want to use a different threshold, e.g., z-score threshold of 2.5:
#outlier_sorted_context = find_outliers_with_context(average_household_size, z_threshold=2.5)

# Display the results
#print(outlier_sorted_context)


In [19]:
def find_ahs_outliers_pct_change(df, pct_change_threshold=0.3):
        """
        Identifies outliers in the average household size (ahs) dataframe based on percentage change and returns a DataFrame with additional context.
        
        Args:
        - df (pd.DataFrame): DataFrame with household size data. First column should be 'gss_code_ward'.
        - pct_change_threshold (float): Threshold for percentage change to determine outliers. Default is 0.2 (20%).
        
        Returns:
        - pd.DataFrame: DataFrame containing gss_code_ward, year, pct_change, outlier value, year_before, year_after,
                        average_for_year (across wards), and average_for_ward (across years).
        """
        
        # Step 1: Calculate percentage change for each year column (excluding 'gss_code_ward')
        pct_changes = df.iloc[:, 1:].pct_change(axis=1)
        
        # Step 2: Identify the outliers (percentage change > pct_change_threshold or < -pct_change_threshold)
        outliers = pct_changes[(pct_changes > pct_change_threshold) | (pct_changes < -pct_change_threshold)]

        # Step 3: Combine 'gss_code_ward' with outliers to present them in a readable format
        outliers_cleaned = pd.concat([df['gss_code_ward'], outliers], axis=1)

        # Step 4: Drop rows with no outliers
        outliers_cleaned = outliers_cleaned.dropna(how='all', subset=outliers_cleaned.columns[1:])

        # Step 5: Melt the DataFrame to have 'gss_code_ward', 'year', and 'pct_change' columns
        outliers_melted = outliers_cleaned.melt(id_vars='gss_code_ward', var_name='year', value_name='pct_change')

        # Step 6: Drop NaN values (non-outliers) and sort by pct_change in descending order
        outliers_sorted = outliers_melted.dropna().sort_values(by='pct_change', ascending=False)

        # Convert the 'year' column to integer for easier manipulation
        outliers_sorted['year'] = outliers_sorted['year'].astype(int)

        # Step 7: Create new columns for the value of AHS for outlier year, previous year, and next year
        outliers_sorted['outlier_value'] = outliers_sorted.apply(
            lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], str(row['year'])].values[0], axis=1)

        outliers_sorted['year_before_value'] = outliers_sorted.apply(
            lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], str(row['year'] - 1)].values[0]
            if str(row['year'] - 1) in df.columns else None, axis=1)

        outliers_sorted['year_after_value'] = outliers_sorted.apply(
            lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], str(row['year'] + 1)].values[0]
            if str(row['year'] + 1) in df.columns else None, axis=1)

        # Step 8: Calculate the average for that year across all gss_code_wards
        outliers_sorted['average_for_year'] = outliers_sorted.apply(
            lambda row: df[str(row['year'])].mean(), axis=1)

        # Step 9: Calculate the average for that gss_code_ward across all years
        outliers_sorted['average_for_ward'] = outliers_sorted.apply(
            lambda row: df.loc[df['gss_code_ward'] == row['gss_code_ward'], df.columns[1:]].mean(axis=1).values[0], axis=1)

        # Step 10: Create the final outlier_sorted_context DataFrame
        outlier_sorted_context = outliers_sorted[['gss_code_ward', 'year', 'pct_change', 'outlier_value', 'year_before_value', 'year_after_value', 'average_for_year', 'average_for_ward']]

        return outlier_sorted_context

    # Example usage:
    # outlier_sorted_context = find_ahs_outliers_with_pct_change(average_household_size, pct_change_threshold=0.2)

    # If you want to use a different threshold, e.g., percentage change threshold of 15%:
    # outlier_sorted_context = find_ahs_outliers_with_pct_change(average_household_size, pct_change_threshold=0.15)

    # Display the results
    # print(outlier_sorted_context)

## Process data

---

In [167]:
#split ward and borough dataframes
combined_10yr_fert_boroughs = combined_10yr_fert[combined_10yr_fert['gss_code_ward'].isna()]
combined_10yr_fert_ward = combined_10yr_fert[~combined_10yr_fert['gss_code_ward'].isna()]

In [168]:
#bin ages
combined_10yr_fert_agebins = create_age_bins(combined_10yr_fert)

In [169]:
#seperate components into columns
combined_10yr_fert_agebins_component_columns = combined_10yr_fert_agebins.pivot_table(index=['gss_code','gss_code_ward','sex', 'age','year'], columns='component', values='value').reset_index()

C:\Users\user\AppData\Local\Temp\ipykernel_27328\2449090281.py:2: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior



## Basic Checks
Perform basic checks on the dataset, including checking for missing values, duplicates, and descriptive statistics.

---

In [170]:
#min and max year
def get_year_range(df):
    return df['year'].max(), df['year'].min()

In [171]:
#year ranges
print(get_year_range(combined_10yr_fert))
print(get_year_range(combined_10yr_fert_ward))
print(get_year_range(combined_10yr_fert_boroughs))

(2050.0, 2002.0)
(2050.0, 2011.0)
(2050.0, 2002.0)


##### missing values

In [172]:
missing_values = combined_10yr_fert.isnull().sum()
print("Missing values per column:\n", missing_values)

Missing values per column:
 gss_code              0
la_name               0
year                  0
sex                   0
age                   0
value                 0
component             0
gss_code_ward    771342
ward_name        771342
dtype: int64


#### duplicates

In [173]:
duplicates = combined_10yr_fert.duplicated().sum()
print("Number of duplicate rows:", duplicates)

Number of duplicate rows: 0


##### Descriptive data

In [174]:
combined_10yr_fert.describe()

,year,age,value
count,1.542806e+07,1.542806e+07,1.542806e+07
mean,2.030736e+03,4.483779e+01,4.879530e+01
std,1.143556e+01,2.635879e+01,2.227471e+02
min,2.002000e+03,0.000000e+00,-7.130000e+02
25%,2.021000e+03,2.200000e+01,0.000000e+00
50%,2.031000e+03,4.500000e+01,8.000000e-01
75%,2.041000e+03,6.800000e+01,4.790000e+01
max,2.050000e+03,9.000000e+01,5.922600e+03


##### Description by components

In [175]:
# Group by 'component' column
grouped = combined_10yr_fert.groupby('component')

# Apply describe to each group
described_groups = grouped.describe()

In [176]:
described_groups

year                                                   \
               count         mean        std     min      25%     50%   
component                                                               
births       55614.0  2031.000000  11.254730  2012.0  2021.00  2031.0   
deaths     5060874.0  2031.000000  11.254630  2012.0  2021.00  2031.0   
netflow    5120934.0  2030.712656  11.499358  2002.0  2021.00  2031.0   
popn       5190640.0  2030.500000  11.543397  2011.0  2020.75  2030.5   

                                  age        ...                  value  \
               75%     max      count  mean  ...   75%   max      count   
component                                    ...                          
births     2041.00  2050.0    55614.0   0.0  ...   0.0   0.0    55614.0   
deaths     2041.00  2050.0  5060874.0  45.0  ...  68.0  90.0  5060874.0   
netflow    2041.00  2050.0  5120934.0  45.0  ...  68.0  90.0  5120934.0   
popn       2040.25  2050.0  5190640.0  45.0  ...  68.0  90.0  5190640.0   

                                                                     
                 mean         std    min   25%   50%    75%     max  
component                                                            
births     162.293288  377.997791    0.0  59.9  81.6  110.9  3264.0  
deaths       0.885798    6.327541    0.0   0.0   0.1    0.6   825.2  
netflow     -0.182117   20.742161 -713.0  -2.3  -0.4    1.2  1938.0  
popn       142.610704  363.229346    0.0  45.0  71.2  103.6  5922.6  

[4 rows x 24 columns]

In [177]:
# List of percentage change columns
pct_change_columns = ['births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change']
component_columns = ['births', 'deaths', 'netflow', 'popn'] 

# Call the function
descriptive_stats = view_descriptive_statistics(combined_10yr_fert_agebins_component_columns, component_columns)

# Display the descriptive statistics
print(descriptive_stats)

component        births         deaths        netflow           popn
count      53040.000000  477360.000000  477360.000000  489600.000000
mean          85.084712       1.491097      -0.239197      70.710902
std           36.143086       4.131203       3.288641      44.500500
min            0.000000       0.000000     -50.050000       0.000000
25%           59.100000       0.021053      -1.220000      37.880000
50%           79.600000       0.220000      -0.470000      65.326316
75%          106.000000       0.910000       0.270000      95.850000
max          615.900000      87.300000      97.825000     737.090000
median        79.600000       0.220000      -0.470000      65.326316
mode          60.000000       0.000000      -0.500000      13.000000


#### check for negative values in columns

In [178]:
# Checking for negative values and extremely high values
negative_values = combined_10yr_fert[combined_10yr_fert['value'] < 0]
print('components with negative values:', negative_values['component'].unique())

components with negative values: ['netflow']


#### check age range

In [179]:
#print true if max age is 90 and min age is 0
print('Is max age is 90 and min age is 0:', (combined_10yr_fert['age'].max() == 90) & (combined_10yr_fert['age'].min() == 0))

Is max age is 90 and min age is 0: True


## Population Consistency Over Time

---

In [180]:
#use binned ages this will even out large flunctions between age group where the are likely to be unusally high e.i. 18 year olds moving to university
combined_10yr_fert_agebins_component_columns

component,gss_code,gss_code_ward,sex,age,year,births,deaths,netflow,popn
0,E09000001,E09000001,female,0-18,2011.0,NaN,NaN,NaN,19.421053
1,E09000001,E09000001,female,0-18,2012.0,32.0,0.0,0.631579,20.368421
2,E09000001,E09000001,female,0-18,2013.0,36.0,0.0,-1.526316,19.947368
3,E09000001,E09000001,female,0-18,2014.0,26.0,0.0,0.052632,19.473684
4,E09000001,E09000001,female,0-18,2015.0,30.0,0.0,-0.210526,19.578947
...,...,...,...,...,...,...,...,...,...
489595,E09000033,E05013809,male,90+,2046.0,NaN,7.8,0.900000,20.000000
489596,E09000033,E05013809,male,90+,2047.0,NaN,8.1,0.900000,21.000000
489597,E09000033,E05013809,male,90+,2048.0,NaN,8.5,0.900000,22.200000
489598,E09000033,E05013809,male,90+,2049.0,NaN,8.7,0.900000,23.200000


In [181]:
outliers_dict_borough = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['births', 'deaths', 'netflow', 'popn'], 
    handle_inf=False, 
    Geography='borough',
    z_score_threshold=3,
    For_population_totals=False
    )

C:\Users\user\AppData\Local\Temp\ipykernel_27328\3628628267.py:43: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\user\AppData\Local\Temp\ipykernel_27328\3628628267.py:43: FutureWarning:

The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



births_pct_change used Robust Z-Score.
deaths_pct_change used Robust Z-Score.
netflow_pct_change used Robust Z-Score.
popn_pct_change used Robust Z-Score.


In [182]:
outliers_dict_ward = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['births', 'deaths', 'netflow', 'popn'], 
    handle_inf=False, 
    Geography='ward',
    z_score_threshold=3,
    For_population_totals=False
    )

C:\Users\user\AppData\Local\Temp\ipykernel_27328\3628628267.py:43: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\user\AppData\Local\Temp\ipykernel_27328\3628628267.py:43: FutureWarning:

The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



births_pct_change used Robust Z-Score.
deaths_pct_change used Robust Z-Score.
netflow_pct_change used Robust Z-Score.
popn_pct_change used Robust Z-Score.


In [183]:
outliers_dict_borough_inf_values = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['births', 'deaths', 'netflow', 'popn'], 
    handle_inf=True, 
    Geography='borough',
    z_score_threshold=3,
    For_population_totals=False
    )

C:\Users\user\AppData\Local\Temp\ipykernel_27328\3628628267.py:43: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\user\AppData\Local\Temp\ipykernel_27328\3628628267.py:43: FutureWarning:

The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



births used Robust Z-Score.
deaths used Robust Z-Score.
netflow used Robust Z-Score.
popn used Robust Z-Score.


In [184]:
outliers_dict_ward_inf_values = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['births', 'deaths', 'netflow', 'popn'], 
    handle_inf=True, 
    Geography='ward',
    z_score_threshold=3,
    For_population_totals=False)

C:\Users\user\AppData\Local\Temp\ipykernel_27328\3628628267.py:43: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

C:\Users\user\AppData\Local\Temp\ipykernel_27328\3628628267.py:43: FutureWarning:

The default fill_method='ffill' in DataFrameGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



births used Robust Z-Score.
deaths used Robust Z-Score.
netflow used Robust Z-Score.
popn used Robust Z-Score.


In [185]:
births_pct_change_borough_outlier_df = outliers_dict_borough['births_pct_change']
deaths_pct_change_borough_outlier_df = outliers_dict_borough['deaths_pct_change']
netflow_pct_change_borough_outlier_df = outliers_dict_borough['netflow_pct_change']
popn_pct_change_borough_outlier_df = outliers_dict_borough['popn_pct_change']

In [186]:
births_pct_change_ward_outlier_df = outliers_dict_ward['births_pct_change']
deaths_pct_change_ward_outlier_df = outliers_dict_ward['deaths_pct_change']
netflow_pct_change_ward_outlier_df = outliers_dict_ward['netflow_pct_change']
popn_pct_change_ward_outlier_df = outliers_dict_ward['popn_pct_change']

In [187]:
births_pct_change_borough_inf_outlier_df = outliers_dict_borough_inf_values['births']
deaths_pct_change_borough_inf_outlier_df = outliers_dict_borough_inf_values['deaths']
netflow_pct_change_borough_inf_outlier_df = outliers_dict_borough_inf_values['netflow']
popn_pct_change_borough_inf_outlier_df = outliers_dict_borough_inf_values['popn']

In [188]:
births_pct_change_ward_inf_outlier_df = outliers_dict_ward_inf_values['births']
deaths_pct_change_ward_inf_outlier_df = outliers_dict_ward_inf_values['deaths']
netflow_pct_change_ward_inf_outlier_df = outliers_dict_ward_inf_values['netflow']   
popn_pct_change_ward_inf_outlier_df = outliers_dict_ward_inf_values['popn']

## Total Population Outliers
#### Detect total outliers using Z-scores and Robust Z-scores, and perform cross-sectional and temporal comparisons.

#### Using total population (popn) perform cross-sectional comparisons: Examine changes between boroughs and wards totals for a given year.
#### Conduct temporal comparisons: Measure percentage changes between year total for both boroughs and wards

#### Total population per geographical boundary
#### Total population per year
#### i.e.
#### groupby('gss_code')['popn']
#### groupby('year')['popn']
---

In [189]:
combined_10yr_fert_agebins_component_columns

component,gss_code,gss_code_ward,sex,age,year,births,deaths,netflow,popn,births_pct_change,deaths_pct_change,netflow_pct_change,popn_pct_change
0,E09000001,E09000001,female,0-18,2011.0,NaN,NaN,NaN,19.421053,NaN,NaN,NaN,NaN
1,E09000001,E09000001,female,0-18,2012.0,32.0,0.0,0.631579,20.368421,NaN,NaN,NaN,0.048780
2,E09000001,E09000001,female,0-18,2013.0,36.0,0.0,-1.526316,19.947368,0.125000,NaN,3.416667,0.020672
3,E09000001,E09000001,female,0-18,2014.0,26.0,0.0,0.052632,19.473684,0.277778,NaN,1.034483,0.023747
4,E09000001,E09000001,female,0-18,2015.0,30.0,0.0,-0.210526,19.578947,0.153846,NaN,5.000000,0.005405
...,...,...,...,...,...,...,...,...,...,...,...,...,...
489595,E09000033,E05013809,male,90+,2046.0,NaN,7.8,0.900000,20.000000,NaN,0.040000,0.000000,0.063830
489596,E09000033,E05013809,male,90+,2047.0,NaN,8.1,0.900000,21.000000,NaN,0.038462,0.000000,0.050000
489597,E09000033,E05013809,male,90+,2048.0,NaN,8.5,0.900000,22.200000,NaN,0.049383,0.000000,0.057143
489598,E09000033,E05013809,male,90+,2049.0,NaN,8.7,0.900000,23.200000,NaN,0.023529,0.000000,0.045045


In [190]:
total_pop_temporal_outliers_ward = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['popn'], 
    handle_inf=False, 
    Geography='ward',
    For_population_totals=True,
    population_analysis_type='temporal'
    )

C:\Users\user\AppData\Local\Temp\ipykernel_27328\3628628267.py:43: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



popn_pct_change used Robust Z-Score.


In [191]:
total_pop_temporal_outliers_ward_df  = total_pop_temporal_outliers_ward['popn_pct_change']

In [192]:
total_pop_cross_sectional_outliers_ward = calculate_zscores_and_find_outliers(
    combined_10yr_fert_agebins_component_columns, 
    ['popn'], 
    handle_inf=False, 
    Geography='ward',
    For_population_totals=True,
    population_analysis_type='cross-sectional'
    )

C:\Users\user\AppData\Local\Temp\ipykernel_27328\3628628267.py:43: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



popn_pct_change used Robust Z-Score.


In [193]:
total_pop_cross_sectional_outliers_ward_df = total_pop_cross_sectional_outliers_ward['popn_pct_change']

## Gender Outliers
Investigate gender outliers, focusing on abnormal gender ratios and adjusting thresholds as needed.

---

In [194]:
#single year age 
component_columns = ['births', 'deaths', 'netflow', 'popn']
gender_outlier_dictionary = gender_outliers(combined_10yr_fert, component_columns, geography='ward', outlier_std={'births': 2, 'deaths': 5, 'netflow': 2, 'popn': 5})

C:\Users\user\AppData\Local\Temp\ipykernel_27328\1554508619.py:40: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\user\AppData\Local\Temp\ipykernel_27328\1554508619.py:41: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [195]:
#binned age
component_columns = ['births', 'deaths', 'netflow', 'popn']
gender_outlier_dictionary = gender_outliers(combined_10yr_fert_agebins, component_columns, geography='ward', outlier_std={'births': 2, 'deaths': 5, 'netflow': 2, 'popn': 5})

C:\Users\user\AppData\Local\Temp\ipykernel_27328\1554508619.py:29: FutureWarning:

The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior

C:\Users\user\AppData\Local\Temp\ipykernel_27328\1554508619.py:40: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



C:\Users\user\AppData\Local\Temp\ipykernel_27328\1554508619.py:41: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill 

In [196]:
gender_outlier_dictionary
births_gender_outliers_df = gender_outlier_dictionary['births']
deaths_gender_outliers_df = gender_outlier_dictionary['deaths']
netflow_gender_outliers_df = gender_outlier_dictionary['netflow']
popn_gender_outliers_df = gender_outlier_dictionary['popn']

In [197]:
births_gender_outliers_df

component,gss_code_ward,year,age,births,robust_z_score
0,E05009317,2012.0,0-18,0.972733,24.599647
1,E05009317,2013.0,0-18,1.139108,225.697260
2,E05009317,2014.0,0-18,1.204861,305.173778
3,E05009317,2015.0,0-18,0.751933,-242.282254
4,E05009317,2016.0,0-18,0.912195,-48.572774
...,...,...,...,...,...
7779,E09000001,2021.0,0-18,0.758621,-234.198805
7780,E09000001,2022.0,0-18,0.894737,-69.674667
7781,E09000001,2023.0,0-18,0.955720,4.035385
7782,E09000001,2039.0,0-18,0.950000,-2.877867


In [198]:
gender_outlier_dictionary

{'births': component gss_code_ward    year   age    births  robust_z_score
 0             E05009317  2012.0  0-18  0.972733       24.599647
 1             E05009317  2013.0  0-18  1.139108      225.697260
 2             E05009317  2014.0  0-18  1.204861      305.173778
 3             E05009317  2015.0  0-18  0.751933     -242.282254
 4             E05009317  2016.0  0-18  0.912195      -48.572774
 ...                 ...     ...   ...       ...             ...
 7779          E09000001  2021.0  0-18  0.758621     -234.198805
 7780          E09000001  2022.0  0-18  0.894737      -69.674667
 7781          E09000001  2023.0  0-18  0.955720        4.035385
 7782          E09000001  2039.0  0-18  0.950000       -2.877867
 7783          E09000001  2049.0  0-18  0.950704       -2.026667
 
 [7784 rows x 5 columns],
 'deaths': component gss_code_ward    year    age    deaths  robust_z_score
 0             E05009317  2013.0    90+  2.760000        5.519995
 1             E05009317  2014.0  31-40 

## Key Visualisations
Visualise important trends in the dataset, including distribution by components, age groups, yearly totals, and population pyramids.

---


#### yearly totals by component visulation

In [199]:
yearly_totals = combined_10yr_fert.groupby(['year','component'])['value'].sum().reset_index()
print("Yearly population totals by gss_code and year:\n", yearly_totals)

Yearly population totals by gss_code and year:
        year component       value
0    2002.0   netflow      7036.6
1    2003.0   netflow    -37021.3
2    2004.0   netflow    -16399.9
3    2005.0   netflow     21802.2
4    2006.0   netflow      9229.4
..      ...       ...         ...
162  2049.0      popn  19763868.6
163  2050.0    births    226639.0
164  2050.0    deaths    137879.9
165  2050.0   netflow    -57727.1
166  2050.0      popn  19794455.5

[167 rows x 3 columns]


In [200]:

# Create a bar graph using Plotly Express
fig = px.line(
    yearly_totals, 
    x='year', 
    y='value', 
    color='component', 
    markers=True,
    title="Yearly Totals by Component Over the Years",
    labels={'value': 'Total Value', 'year': 'Year'},
    
)

# Update layout for better appearance
fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Total Value',
    legend_title='Component',
    width=900,
    height=600
)

# Show the figure
fig.show()

#### population pyramid

In [201]:
#for unit age
combined_10yr_fert_popn = combined_10yr_fert[combined_10yr_fert['component'] == 'popn']
population_pyramids_unit_age = combined_10yr_fert_popn.copy()
population_pyramids_unit_age = population_pyramids_unit_age[~population_pyramids_unit_age['gss_code_ward'].isna()]

In [202]:
population_pyramids_unit_age

,gss_code,la_name,year,sex,age,value,component,gss_code_ward,ward_name
10477662,E09000001,City of London,2011.0,female,0.0,37.0,popn,E09000001,City of London
10477663,E09000001,City of London,2011.0,female,1.0,34.0,popn,E09000001,City of London
10477664,E09000001,City of London,2011.0,female,2.0,28.0,popn,E09000001,City of London
10477665,E09000001,City of London,2011.0,female,3.0,18.0,popn,E09000001,City of London
10477666,E09000001,City of London,2011.0,female,4.0,21.0,popn,E09000001,City of London
...,...,...,...,...,...,...,...,...,...
15428057,E09000033,Westminster,2050.0,male,86.0,16.9,popn,E05013809,Westbourne
15428058,E09000033,Westminster,2050.0,male,87.0,13.3,popn,E05013809,Westbourne
15428059,E09000033,Westminster,2050.0,male,88.0,11.0,popn,E05013809,Westbourne
15428060,E09000033,Westminster,2050.0,male,89.0,9.9,popn,E05013809,Westbourne


In [223]:
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        id='location-dropdown',
        options=[{'label': loc, 'value': loc} for loc in population_pyramids_unit_age['la_name'].unique()] + [{'label': 'London Total (All LAs)', 'value': 'London Total (All LAs)'}],
        value=population_pyramids_unit_age['la_name'].unique()[0]
    ),
    dcc.Dropdown(
        id='ward-dropdown'
    ),
    dcc.Graph(id='population-pyramid')
])

@app.callback(
    Output('ward-dropdown', 'options'),
    Output('ward-dropdown', 'value'),
    Input('location-dropdown', 'value')
)
def update_ward_dropdown(selected_location):
    if selected_location == 'London Total (All LAs)':
        # If "London Total (All LAs)" is selected, disable the ward dropdown
        return [{'label': 'All Wards', 'value': 'All Wards'}], 'All Wards'
    else:
        # Filter data for the selected location to get ward names
        filtered_data = population_pyramids_unit_age[population_pyramids_unit_age['la_name'] == selected_location]
        
        # Get unique ward names for the selected location
        ward_options = [{'label': ward, 'value': ward} for ward in filtered_data['ward_name'].unique()]
        
        # Add 'All Wards' option
        ward_options.insert(0, {'label': 'All Wards', 'value': 'All Wards'})
        
        # Return the options and set the default value to 'All Wards'
        return ward_options, 'All Wards'

@app.callback(
    Output('population-pyramid', 'figure'),
    Input('location-dropdown', 'value'),
    Input('ward-dropdown', 'value')
)
def update_pyramid(selected_location, selected_ward):
    if selected_location == 'London Total (All LAs)':
        # Combine data for all locations
        filtered_data = population_pyramids_unit_age.copy()
        title = 'Population Pyramid for London Total (All LAs)'
    elif selected_ward == 'All Wards':
        # Combine data for all wards in the selected location
        filtered_data = population_pyramids_unit_age[population_pyramids_unit_age['la_name'] == selected_location]
        title = f'Population Pyramid for {selected_location} - All Wards'
    else:
        # Filter data for the selected location and ward
        filtered_data = population_pyramids_unit_age[
            (population_pyramids_unit_age['la_name'] == selected_location) & 
            (population_pyramids_unit_age['ward_name'] == selected_ward)
        ]
        title = f'Population Pyramid for {selected_location} - {selected_ward}'

    # Negate female population values to create a pyramid
    filtered_data['value'] = filtered_data.apply(lambda row: -row['value'] if row['sex'] == 'female' else row['value'], axis=1)

    # Create a plotly express bar chart with animation
    fig = px.bar(
        filtered_data,
        x='value',
        y='age',
        color='sex',
        animation_frame='year',
        orientation='h',
        title=title,
        labels={'value': 'Population', 'age': 'Age'},
        color_discrete_map={'male': 'blue', 'female': 'pink'},
        height=600,
        range_x=[-filtered_data['value'].max()*1.2, filtered_data['value'].max()*1.2], #need to keep x-axis consistent over time for comparison
        hover_data={'ward_name': True}
    )

    # Update layout for better appearance
    fig.update_layout(
        barmode='relative',
        xaxis_title='Population',
        yaxis_title='Age',
        showlegend=True,
        width=800,
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=1223)

C:\Users\user\AppData\Local\Temp\ipykernel_27328\668186827.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\user\AppData\Local\Temp\ipykernel_27328\668186827.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\user\AppData\Local\Temp\ipykernel_27328\668186827.py:60: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-d


## Collate Outliers
Summarize and compile all detected outliers for further analysis.

---

In [204]:
# Filter the global variables to find dataframes with 'outlier' in their name
outlier_dfs = {name: df for name, df in globals().items() if isinstance(df, pd.DataFrame) and 'outlier' in name.lower()}

# Display the name, columns, and length of each dataframe
for name, df in outlier_dfs.items():
    print(f"DataFrame Name: {name}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Length: {len(df)}")
    print("\n" + "-"*50 + "\n")

DataFrame Name: births_pct_change_borough_outlier_df
Columns: ['gss_code', 'gss_code_ward', 'sex', 'age', 'year', 'births', 'deaths', 'netflow', 'popn', 'births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change', 'robust_z_score']
Length: 16305

--------------------------------------------------

DataFrame Name: deaths_pct_change_borough_outlier_df
Columns: ['gss_code', 'gss_code_ward', 'sex', 'age', 'year', 'births', 'deaths', 'netflow', 'popn', 'births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change', 'robust_z_score']
Length: 117296

--------------------------------------------------

DataFrame Name: netflow_pct_change_borough_outlier_df
Columns: ['gss_code', 'gss_code_ward', 'sex', 'age', 'year', 'births', 'deaths', 'netflow', 'popn', 'births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change', 'robust_z_score']
Length: 138078

--------------------------------------------------

DataFrame Name: popn_pct_change_bo

In [205]:
# Only ward data datatframes
# Filter the global variables to find dataframes with 'outlier' in their name and exclude ones with 'borough'
outlier_dfs_wards = {name: df for name, df in globals().items() 
               if isinstance(df, pd.DataFrame) 
               and 'outlier' in name.lower() 
               and 'borough' not in name.lower()}
# Display the name, columns, and length of each dataframe
for name, df in outlier_dfs_wards.items():
    print(f"DataFrame Name: {name}")
    print(f"Columns: {df.columns.tolist()}")
    print(f"Length: {len(df)}")
    print("\n" + "-"*50 + "\n")

DataFrame Name: births_pct_change_ward_outlier_df
Columns: ['gss_code', 'gss_code_ward', 'sex', 'age', 'year', 'births', 'deaths', 'netflow', 'popn', 'births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change', 'robust_z_score']
Length: 15207

--------------------------------------------------

DataFrame Name: deaths_pct_change_ward_outlier_df
Columns: ['gss_code', 'gss_code_ward', 'sex', 'age', 'year', 'births', 'deaths', 'netflow', 'popn', 'births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change', 'robust_z_score']
Length: 107953

--------------------------------------------------

DataFrame Name: netflow_pct_change_ward_outlier_df
Columns: ['gss_code', 'gss_code_ward', 'sex', 'age', 'year', 'births', 'deaths', 'netflow', 'popn', 'births_pct_change', 'deaths_pct_change', 'netflow_pct_change', 'popn_pct_change', 'robust_z_score']
Length: 129644

--------------------------------------------------

DataFrame Name: popn_pct_change_ward_outlie

In [206]:
# View age column in dfs
age_snippets = {df_name: df['age'].iloc[0] for df_name, df in outlier_dfs_wards.items()}
age_snippets

{'births_pct_change_ward_outlier_df': '0-18',
 'deaths_pct_change_ward_outlier_df': '41-50',
 'netflow_pct_change_ward_outlier_df': '81-89',
 'popn_pct_change_ward_outlier_df': '90+',
 'births_pct_change_ward_inf_outlier_df': '0-18',
 'deaths_pct_change_ward_inf_outlier_df': '90+',
 'netflow_pct_change_ward_inf_outlier_df': '19-30',
 'popn_pct_change_ward_inf_outlier_df': '31-40',
 'total_pop_temporal_outliers_ward_df': '90+',
 'total_pop_cross_sectional_outliers_ward_df': '90+',
 'births_gender_outliers_df': '0-18',
 'deaths_gender_outliers_df': '90+',
 'netflow_gender_outliers_df': '81-89',
 'popn_gender_outliers_df': '81-89'}

In [207]:
# Initialise an empty list to store relevant rows from all DataFrames
all_rows = []

# Loop through all outlier dataframes
for name, df in outlier_dfs_wards.items():
    # Check if 'gss_code' and 'year' columns exist
    if 'gss_code_ward' in df.columns and 'year' in df.columns:
        # Select the relevant columns: 'gss_code', 'year' and 'age' (if it exists)
        cols = ['gss_code_ward', 'year']
        if 'age' in df.columns:
            cols.append('age')

        # Append the relevant data from the current DataFrame to the list
        all_rows.append(df[cols])

# Concatenate all collected data into one DataFrame
if all_rows:
    combined_df = pd.concat(all_rows)

    # Group by 'gss_code', 'year', and 'age' (where applicable) and count occurrences
    tally_df = combined_df.groupby(cols).size().reset_index(name='count')

    # Sort the result by count in descending order
    tally_df = tally_df.sort_values(by='count', ascending=False)

    # Display the top rows of the tally DataFrame
    print(tally_df)
else:
    print("No relevant data found.")

C:\Users\user\AppData\Local\Temp\ipykernel_27328\469539181.py:21: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



       gss_code_ward    year    age  count
221211     E05014055  2030.0   0-18     17
204956     E05013988  2023.0    90+     17
204866     E05013988  2013.0    90+     17
109547     E05013651  2022.0    90+     17
109187     E05013650  2022.0    90+     17
...              ...     ...    ...    ...
134537     E05013720  2039.0  61-70      0
134536     E05013720  2039.0  51-60      0
134534     E05013720  2039.0  31-40      0
134533     E05013720  2039.0  19-30      0
122400     E05013687  2011.0   0-18      0

[244800 rows x 4 columns]


In [208]:
outlier_dfs_wards

{'births_pct_change_ward_outlier_df': component   gss_code gss_code_ward     sex   age    year  births    deaths  \
 465485     E09000032     E05014015    male  0-18  2016.0     3.3  0.000000   
 465122     E09000032     E05014015  female  0-18  2013.0     1.1  0.000000   
 368644     E09000025     E05013925  female  0-18  2015.0    40.9  0.063158   
 369004     E09000025     E05013925    male  0-18  2015.0    43.9  0.000000   
 368643     E09000025     E05013925  female  0-18  2014.0     5.0  0.047368   
 ...              ...           ...     ...   ...     ...     ...       ...   
 460449     E09000031     E05013903    male  0-18  2020.0    85.2  0.000000   
 393136     E09000027     E05013783  female  0-18  2027.0    85.2  0.010526   
 100460     E09000008     E05011476    male  0-18  2031.0   116.6  0.021053   
 3266       E09000002     E05014056    male  0-18  2037.0   120.0  0.021053   
 356777     E09000025     E05013908    male  0-18  2028.0   134.6  0.031579   
 
 component   

In [209]:
df_tally = tally_outliers(outlier_dfs_wards, tally_by_age=True)


In [210]:
df_tally

,gss_code_ward,year,age,births_pct_change_ward_outlier_df,deaths_pct_change_ward_outlier_df,netflow_pct_change_ward_outlier_df,popn_pct_change_ward_outlier_df,births_pct_change_ward_inf_outlier_df,deaths_pct_change_ward_inf_outlier_df,netflow_pct_change_ward_inf_outlier_df,popn_pct_change_ward_inf_outlier_df,total_pop_temporal_outliers_ward_df,total_pop_cross_sectional_outliers_ward_df,births_gender_outliers_df,deaths_gender_outliers_df,netflow_gender_outliers_df,popn_gender_outliers_df,total,total_outlier_score
0,E05013631,2013.0,90+,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,11.0,254.683507
1,E05011255,2013.0,90+,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,11.0,162.486896
2,E05013988,2014.0,90+,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,11.0,343.900304
3,E05013988,2013.0,90+,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,11.0,335.147998
4,E05011113,2020.0,90+,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,10.0,92.180257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184560,E05013788,2017.0,41-50,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20.663021
184561,E05013511,2012.0,41-50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-5.286677
184562,E05013511,2012.0,0-18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,249.527969
184563,E05013788,2018.0,41-50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.074074


In [211]:
df_tally

,gss_code_ward,year,age,births_pct_change_ward_outlier_df,deaths_pct_change_ward_outlier_df,netflow_pct_change_ward_outlier_df,popn_pct_change_ward_outlier_df,births_pct_change_ward_inf_outlier_df,deaths_pct_change_ward_inf_outlier_df,netflow_pct_change_ward_inf_outlier_df,popn_pct_change_ward_inf_outlier_df,total_pop_temporal_outliers_ward_df,total_pop_cross_sectional_outliers_ward_df,births_gender_outliers_df,deaths_gender_outliers_df,netflow_gender_outliers_df,popn_gender_outliers_df,total,total_outlier_score
0,E05013631,2013.0,90+,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,11.0,254.683507
1,E05011255,2013.0,90+,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,11.0,162.486896
2,E05013988,2014.0,90+,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,11.0,343.900304
3,E05013988,2013.0,90+,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,11.0,335.147998
4,E05011113,2020.0,90+,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,10.0,92.180257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184560,E05013788,2017.0,41-50,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,20.663021
184561,E05013511,2012.0,41-50,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,-5.286677
184562,E05013511,2012.0,0-18,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,249.527969
184563,E05013788,2018.0,41-50,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.074074


In [212]:
# Concatenate all the DataFrames together
combined_df = pd.concat(outlier_dfs_wards)

# Group by 'gss_code_ward', 'year', and 'age', and sum the outlier scores
grouped_df = combined_df.groupby(['gss_code_ward', 'year', 'age'], as_index=False).agg(
    total_outlier_score=('robust_z_score', 'sum')
)

In [213]:
grouped_df

,gss_code_ward,year,age,total_outlier_score
0,E05009317,2012.0,0-18,24.599647
1,E05009317,2012.0,19-30,28.160639
2,E05009317,2012.0,31-40,11.828891
3,E05009317,2012.0,51-60,14.562207
4,E05009317,2012.0,61-70,4.056622
...,...,...,...,...
184560,E09000001,2050.0,19-30,24.846667
184561,E09000001,2050.0,31-40,-6.993333
184562,E09000001,2050.0,41-50,7.924344
184563,E09000001,2050.0,81-89,14.550325


In [214]:
grouped_df
grouped_df.sort_values(by='total_outlier_score', ascending=False)

,gss_code_ward,year,age,total_outlier_score
129857,E05013826,2014.0,81-89,2.572716e+18
151370,E05013978,2015.0,81-89,2.216494e+18
147480,E05013943,2020.0,51-60,1.899852e+18
166033,E05014053,2016.0,81-89,1.543630e+18
6942,E05009371,2014.0,81-89,1.464469e+18
...,...,...,...,...
112680,E05013759,2012.0,71-80,-1.751040e+17
147766,E05013944,2026.0,71-80,-1.792732e+17
154366,E05013989,2015.0,81-89,-1.834423e+17
129848,E05013826,2013.0,81-89,-2.001189e+17


In [215]:
# potential alterations:
# using single year of age vs binned ages 
# add weighting to final tally
# remove certain age groups like 90+ from outlier detection
# household data
# hmtl report generation


## Average Household Size Outliers
This section will use the function to produce a dataframe with outliers regrading the average household size.
It also contains some contextual results so comparison can be more easily made, such as the year before and after, so that teh change can be seen. As well as the average household size for that word as well as that year across all wards.

In [6]:
#read in rds file
average_household_size = pyreadr.read_r(r"C:\Users\user\Documents\population_data\2022_Identified_Capacity_15yr_high_fert\2022_Identified_Capacity_15yr_high_fert\ahs.rds")
average_household_size = average_household_size[None]

In [7]:
#remove nan columns
average_household_size = average_household_size.dropna(axis=1, how='all')

In [8]:
average_household_size

,gss_code_ward,2021,2022,2023,2024,2025,2026,2027,2028,2029,...,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,E09000001,1.729055,2.141426,2.087426,2.040820,2.014128,1.994125,1.979868,1.968333,1.958878,...,1.910785,1.920089,1.927280,1.933181,1.938368,1.942776,1.946773,1.950291,1.953340,1.956169
1,E05014053,2.813790,2.548360,2.275661,2.153346,2.133935,2.105798,2.090412,2.078165,2.067971,...,2.135847,2.145677,2.156537,2.168107,2.179532,2.191080,2.202550,2.213891,2.224858,2.235705
2,E05014054,2.909020,2.883138,2.922277,2.923762,2.903617,2.882361,2.865726,2.849186,2.833638,...,2.799580,2.807506,2.818172,2.830171,2.843024,2.855550,2.867720,2.879492,2.890862,2.901789
3,E05014055,2.951577,2.634936,2.445635,2.358832,2.298595,2.248707,2.224863,2.213345,2.205129,...,2.170113,2.238887,2.279179,2.306654,2.327505,2.345364,2.360961,2.375091,2.388186,2.400488
4,E05014056,3.048425,2.873637,2.762628,2.698691,2.621724,2.577278,2.570153,2.579542,2.593865,...,2.627652,2.621036,2.617148,2.614639,2.612952,2.611757,2.610252,2.608712,2.606719,2.604483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,E05013805,2.090312,2.182428,2.214119,2.199514,2.189446,2.177585,2.166442,2.155516,2.145647,...,2.083577,2.088475,2.093266,2.097676,2.101798,2.105602,2.109047,2.112118,2.114787,2.117179
676,E05013806,1.805705,1.890946,1.862247,1.822636,1.809524,1.797709,1.787583,1.779312,1.772270,...,1.716521,1.720707,1.725001,1.728974,1.732693,1.735995,1.739187,1.741991,1.744623,1.747075
677,E05013807,2.004284,2.053970,2.069157,2.044950,2.034003,2.020535,2.006764,1.992909,1.982163,...,1.908214,1.912048,1.916078,1.919994,1.923638,1.926997,1.930111,1.932949,1.935549,1.937979
678,E05013808,1.816990,1.941948,1.968739,1.952043,1.940812,1.928824,1.917613,1.907717,1.899374,...,1.845124,1.848929,1.853117,1.857303,1.861331,1.865219,1.868803,1.872123,1.875104,1.877877


In [218]:
# Example usage:
anomalous_average_household_size_sorted_context = find_ahs_outliers_with_context(average_household_size, z_threshold=3)



c:\Users\user\anaconda3\envs\env\Lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning:

invalid value encountered in reduce

c:\Users\user\anaconda3\envs\env\Lib\site-packages\numpy\core\_methods.py:152: RuntimeWarning:

invalid value encountered in reduce

c:\Users\user\anaconda3\envs\env\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning:

invalid value encountered in subtract

c:\Users\user\anaconda3\envs\env\Lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning:

invalid value encountered in reduce

c:\Users\user\anaconda3\envs\env\Lib\site-packages\numpy\core\_methods.py:152: RuntimeWarning:

invalid value encountered in reduce

c:\Users\user\anaconda3\envs\env\Lib\site-packages\numpy\core\_methods.py:173: RuntimeWarning:

invalid value encountered in subtract

c:\Users\user\anaconda3\envs\env\Lib\site-packages\numpy\core\_methods.py:118: RuntimeWarning:

invalid value encountered in reduce

c:\Users\user\anaconda3\envs\env\Lib\site-packages\numpy\core\_me

In [219]:
anomalous_average_household_size_sorted_context

,gss_code_ward,year,z_score,outlier_value,year_before_value,year_after_value,average_for_year,average_for_ward
164,E05013921,2023,4.354520,4.086808,4.036352,4.074413,2.574048,3.821120
151,E05013921,2022,4.335116,4.036352,3.995211,4.086808,2.552756,3.821120
177,E05013921,2024,4.311738,4.074413,4.086808,4.045974,2.562898,3.821120
190,E05013921,2025,4.257276,4.045974,4.074413,4.007388,2.551880,3.821120
203,E05013921,2026,4.195264,4.007388,4.045974,3.971685,2.535572,3.821120
...,...,...,...,...,...,...,...,...
302,E05013538,2034,3.001468,3.497292,3.509477,3.489739,2.454628,3.508954
207,E05014015,2026,-3.018934,1.476446,1.519301,1.454275,2.535572,1.488101
246,E05014015,2029,-3.037740,1.432531,1.441933,1.453046,2.487824,1.488101
220,E05014015,2027,-3.042164,1.454275,1.476446,1.441933,2.519411,1.488101


In [220]:
#unique values in gss_code_ward for outlier_sorted_context
print("outlier wards:", anomalous_average_household_size_sorted_context['gss_code_ward'].unique())

outlier wards: ['E05013921' 'E05013913' 'E05013914' 'E05013537' 'E05011248' 'E05013514'
 'E05013926' 'E05011240' 'E05013539' 'E05013538' 'E05014064' 'E05013909'
 'E05014015']


In [18]:
outlier_sorted_context_pct = find_ahs_outliers_with_pct_change(average_household_size, pct_change_threshold=0.10
                                                               )

In [16]:
outlier_sorted_context_pct

,gss_code_ward,year,pct_change,outlier_value,year_before_value,year_after_value,average_for_year,average_for_ward
11,E05013516,2022,0.335970,2.959530,2.215267,2.972539,2.552756,2.978746
8,E09000001,2022,0.238495,2.141426,1.729055,2.087426,2.552756,1.949733
12,E05013653,2022,0.232125,2.261578,1.835510,2.280820,2.552756,2.107772
15,E05013796,2022,0.128527,2.381155,2.109967,2.354674,2.552756,2.190323
13,E05013570,2022,0.114253,3.213425,2.883927,3.267856,2.552756,3.219093
14,E05009336,2022,0.102414,2.673119,2.424788,2.648123,2.552756,2.491788
17,E05014053,2023,-0.107010,2.275661,2.548360,2.153346,2.574048,2.171645
10,E05014055,2022,-0.107278,2.634936,2.951577,2.445635,2.552756,2.265520
